In [4]:
import jieba
import numpy as np
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [15]:
from os import listdir
from os.path import isfile, join,splitext
t_dir = 'Stage_2\dataPublicComplete_s2\dataPublicComplete'
txt_fnames = [splitext(f)[0] for f in listdir(t_dir) if isfile(join(t_dir, f))] #Article filenames

In [16]:
crop_list = open('Keywords/02crop.list.csv', "r",encoding='UTF-8-sig')
crop = crop_list.read()
crop_line_sep = crop.splitlines()

pest_list = open('Keywords/02pest.list.csv', "r",encoding='UTF-8-sig')
pest = pest_list.read()
pest_line_sep = pest.splitlines()

chem_list = open('Keywords/02chem.list.csv', "r",encoding='UTF-8-sig')
chem = chem_list.read()
chem_line_sep = chem.splitlines()
#Keywords split by lines, keyword with more than one entry will be on the same line

In [17]:
from itertools import chain
import csv
vector_dict = {}
#Keyword lookup with keyword as key and vector index as value
for idx,line in enumerate(chain(crop_line_sep,pest_line_sep,chem_line_sep)):
    l = line.split(',')
    for word in l:
        #Some line will have more than one entry, which should have the same vector index
        if(word == ''):continue
        jieba.add_word(word)#Each keyword is added to jieba
        vector_dict[word] = idx

In [18]:
vectors = {}
#Vector table with filename as key and vector as value
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    vectors[fname] = [False]*764
    #Initialize an 0 vector for each file
    for seg in seg_list:
        if(seg in vector_dict):
            vectors[fname][vector_dict[seg]] = True
            #Count keyword appearence - Bag of words
for key in vectors:
    v = vectors[key]
    curr = 0
    for b in v:
        if(b):curr = (curr << 1) + 1
        else:curr = curr << 1
    vectors[key] = curr

In [19]:
out = []
for Ref in txt_fnames:
    for Test in txt_fnames:
        if(Ref == Test):continue
        if((vectors[Test] ^ (vectors[Test] & vectors[Ref]))== 0 and vectors[Test] > 0 and vectors[Ref] > 0):
            out.append((Test,Ref))

In [20]:
print(len(out))

979


In [22]:
import csv
print(len(out))
with open('val_super_simple.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["Test"]+["Reference"])
    for row in out:  
        spamwriter.writerow(row)

979


In [35]:
txt = open(t_dir+'/'+'574'+'.txt', "r",encoding="utf-8")
content = txt.read()
seg_list = jieba.cut(content, cut_all=True)
tmp = {}
for seg in seg_list:
        if(seg in vector_dict):
            tmp[seg] = vector_dict[seg]
txt = open(t_dir+'/'+'1008'+'.txt', "r",encoding="utf-8")
content = txt.read()
seg_list = jieba.cut(content, cut_all=True)
tmp2 = {}
for seg in seg_list:
        if(seg in vector_dict):
            tmp2[seg] = vector_dict[seg]

In [36]:
for key in vectors:
    if(vectors[key])

({'水稻': 1,
  '縱捲葉蟲': 202,
  '捲葉蟲': 202,
  '瘤野螟': 202,
  '歐殺松': 482,
  '歐殺松可溶性粉劑': 482,
  '陶斯松': 424,
  '陶斯松乳劑': 424},
 {'水稻': 1,
  '縱捲葉蟲': 202,
  '捲葉蟲': 202,
  '瘤野螟': 202,
  '歐殺松': 482,
  '歐殺松可溶性粉劑': 482,
  '陶斯松': 424,
  '陶斯松乳劑': 424})

In [44]:
tmp = {}
for fname in txt_fnames:
    txt = open(t_dir+'/'+fname+'.txt', "r",encoding="utf-8")
    content = txt.read()
    seg_list = jieba.cut(content, cut_all=True)
    t = set()
    ad = False
    for seg in seg_list:
        if(seg in vector_dict):
            t.add(seg)
        if(seg == '台北'):
            ad = True
    if(ad):
        tmp[fname] = [t,content]
for k in tmp:
    print(k,tmp[k][1])

901 發佈水稻螟蟲危害警報
警報發佈內容一、病蟲害別：螟蟲（一點螟、二化螟、大螟）二、防治地點及時期：發生地區：桃園縣、及台北、新竹縣(市)各鄉鎮水稻栽培區防治時期：螟蛾盛發後七天或水稻孕穗後期葉鞘黃化或白穗時。三、防治方法：(一)水稻收穫後儘速翻耕稻田，深埋水稻殘株，並避免堆積稻蒿，以減少螟蟲之越冬場所。(二)栽培抗蟲品種：粳稻品種如台粳14、台粳9號、桃園1號等，比秈稻品種如台中秈10號，發生較輕。(三)避免偏施氮肥：多施氮肥造成稻桿粗大，莖質嫩綠，有利於幼蟲發育，受害較重。(四)藥劑防治方法：於螟蛾盛發後7天或孕穗後期使用下列一種藥劑。1.6%培丹粒劑，每公頃施用30公斤。2.0.3%芬普尼粒劑，每公頃施用20公斤。3.3%加保扶粒劑，每公頃施用40-50公斤。4.50%撲滅松乳劑1,000倍。5.50%培丹可溶性粉劑1,000倍註：施粒劑時田面須保持3公分水位，維持5天。/fontdivid=filelink1fontsize=2face="新細明體"請點選檔名開啟或下載(依檔案大小開啟速度不同)：


903 發佈水稻水象鼻蟲、縱捲葉蟲、螟蟲警報
一、病蟲害別：水象鼻蟲、縱捲葉蟲、螟蟲。二、防治地點：台北、新竹縣(市)及桃園縣各鄉鎮水稻栽培區防治時期：插秧前24小時內或插秧後生育初期如發現水象鼻蟲成蟲入侵時；縱捲葉蛾盛發後七天或稻葉初捲時；螟蛾盛發後七天或水稻葉鞘黃化，葉片下垂或枯心時。三、發生與防治：二期水稻插秧後生育初期容易遭受蟲害，如棲息在鄰近稻田殘株或附近草叢中的水象鼻蟲成蟲、縱捲葉蛾、螟蛾等隨著水稻幼苗成長入侵產卵繁衍危害。防治方法：1.鄰近休耕田水稻殘株儘速翻犁深理。2.稻田附近勿堆積稻蒿，並清除田埂雜草。3.稻田插秧後，田面保持淺水灌溉(0.5公分)，以減少水象鼻蟲成蟲在水面下葉鞘產卵機會。4.適時施用下列藥劑或參考植物保護手冊方法防治：(1)3%加保扶粒劑每箱秧苗50公克，於機械插秧24小時內均勻撒於育苗箱秧苗上，並立即灑水。(2)6%培丹粒劑，每公頃30公斤或0.3%芬普尼粒劑，每公頃20公斤，施藥前田面先灌水1-3公分，再將進(出)水口封住，然後將粒劑均勻撒佈。


908 發佈蓮花小黃薊馬、斜紋夜盜蟲蟲害疫情警報
一、病蟲害別：蓮花小黃薊馬、斜紋夜盜蟲。二、防治地點：桃園縣、及台北、新竹縣(市)各鄉鎮蓮花、田菁綠肥栽培區防治時期：植